In [2]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from imblearn.under_sampling import InstanceHardnessThreshold, RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import IMBS, CLFS, ENSEMBLES, CV, SCORERS
from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob,FilteringEstimator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
class IHFilter(BaseEstimator, ClassifierMixin):
    def __init__(self,estimator,threshold=.5):
        self.estimator = estimator
        self.threshold = threshold
        
    def clean(self,X,Y,sample_weight):
        skf = StratifiedKFold(n_splits=5, shuffle=True)
        rf = RandomForestClassifier(n_estimators=50,n_jobs=4)
        
        probabilities = np.zeros(Y.shape[0], dtype=float)
        for train_index, test_index in skf.split(X,Y):
            rf.fit(X[train_index], Y[train_index])
            probs = rf.predict_proba(X[test_index])
            probabilities[test_index] = probs[range(len(test_index)), Y[test_index]]
            
        hardness = 1 - probabilities
        clean_idx = hardness <= self.threshold
        
        try:
            sample_weight = sample_weight[clean_idx]
        except:
            pass
        Xt,Yt = X[clean_idx],Y[clean_idx]
        return Xt,Yt,sample_weight
    
    def fit(self, X, Y,sample_weight=None):
        Xf,Yf,sample_weight = self.clean(X, Y, sample_weight)
#         a,b = np.unique(Y,return_counts=True)[1],np.unique(Yf,return_counts=True)[1]
#         print(a.max()/a.min(),b.max()/b.min(),a,b,Xf.shape,len(Xf)/len(X))
        assert len(np.unique(Yf))==2,"Pos class completely filtered out"
        try:
            self.estimator = self.estimator.fit(Xf, Yf,sample_weight=sample_weight)
        except TypeError as e:
            self.estimator = self.estimator.fit(Xf, Yf)
        return self

    @property
    def classes_(self):
        return self.estimator.classes_

    def predict(self, X):
        return self.estimator.predict(X)
    
    def predict_proba(self, X):
        return self.estimator.predict_proba(X) 

In [5]:
models = {}
for im,samp in IMBS.items():
    for c,clf in CLFS.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',IHFilter(clf))])
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'svm'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'svm'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'svm'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'svm'), ('tomek', 'knn'), ('tomek', 'rf'), ('None', 'dt'), ('None', 'lr'), ('None', 'nb'), ('None', 'svm'), ('None', 'knn'), ('None', 'rf')]),
 30)

In [6]:
cols = pd.MultiIndex.from_product([IMBS.keys(),CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
#df = pd.read_csv("IHFilter.csv",header=[0,1,2],index_col=0)

In [ ]:
%%time
for it,d in enumerate(DATASETS):
    X,y_noisy,y_real = read_data(d,stats=True)
    if df.loc[d,:].isna().sum()==0:
        print(f"SKIPPING {d}\n")
        continue
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv("IHFilter.csv")

activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('smote', 'dt')
80.03 [0.186, 0.266]
('smote', 'lr')
110.26 [0.255, 0.303]
('smote', 'nb')
59.44 [0.284, 0.402]
('smote', 'svm')
202.99 [0.246, 0.198]
('smote', 'knn')
46.85 [0.205, 0.294]
('smote', 'rf')
57.38 [0.268, 0.282]
('rus', 'dt')
31.73 [0.234, 0.469]
('rus', 'lr')
33.22 [0.258, 0.309]
('rus', 'nb')
30.23 [0.271, 0.457]
('rus', 'svm')
31.73 [0.251, 0.239]
('rus', 'knn')
31.49 [0.239, 0.411]
('rus', 'rf')
31.56 [0.255, 0.326]
('wilson', 'dt')
36.67 [0.233, 0.302]
('wilson', 'lr')
42.51 [0.277, 0.315]
('wilson', 'nb')
39.92 [0.296, 0.401]
('wilson', 'svm')
45.68 [0.301, 0.294]
('wilson', 'knn')
45.31 [0.266, 0.314]
('wilson', 'rf')
45.27 [0.271, 0.33]
('tomek', 'dt')
41.61 [0.258, 0.39]
('tomek', 'lr')
44.43 [0.257, 0.344]
('tomek', 'nb')
43.71 [0.304, 0.351]
('tomek', 'svm')
39.09 [0.264, 0.219]
('tomek', 'knn')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


34.38 [0.238, 0.382]
('tomek', 'rf')
35.3 [0.283, 0.34]
('None', 'dt')
28.09 [0.264, 0.409]
('None', 'lr')
29.85 [0.255, 0.332]
('None', 'nb')
27.82 [0.307, 0.346]
('None', 'svm')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


29.6 [0.255, 0.208]
('None', 'knn')


...

29.39 [0.077, 0.266]
('None', 'rf')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


ERROR!!! Pos class completely filtered out


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

29.32 [0.12, 0.179]

lucene-3.1.csv noise:0.120, imb:7.477,331,2475, Shape:(2806, 65)
('smote', 'dt')
43.41 [0.158, 0.285]
('smote', 'lr')
50.76 [0.163, 0.233]
('smote', 'nb')
41.54 [0.184, 0.27]
('smote', 'svm')
115.72 [0.158, 0.135]
('smote', 'knn')
45.16 [0.149, 0.284]
('smote', 'rf')
52.34 [0.197, 0.155]
('rus', 'dt')
27.75 [0.154, 0.367]
('rus', 'lr')
29.23 [0.169, 0.219]
('rus', 'nb')
27.61 [0.191, 0.298]
('rus', 'svm')
28.46 [0.16, 0.139]
('rus', 'knn')
28.08 [0.162, 0.318]
('rus', 'rf')
29.36 [0.156, 0.243]
('wilson', 'dt')
33.81 [0.175, 0.307]
('wilson', 'lr')
36.74 [0.2, 0.228]
('wilson', 'nb')
34.02 [0.192, 0.279]
('wilson', 'svm')
38.26 [0.189, 0.174]
('wilson', 'knn')
35.35 [0.173, 0.247]
('wilson', 'rf')
37.37 [0.188, 0.185]
('tomek', 'dt')
33.13 [0.16, 0.226]
('tomek', 'lr')
37.78 [0.185, 0.209]
('tomek', 'nb')
34.64 [0.175, 0.264]
('tomek', 'svm')
42.61 [0.165, 0.178]
('tomek', 'knn')
34.93 [0.148, 0.161]
('tomek', 'rf')
37.21 [0.196, 0.184]
('None', 'dt')
29.61 [0.17, 